# Source Localization Workflow

In [2]:
# %pip install nibabel
# %pip install pyvistaqt
# %pip install ipywidgets
# %pip install ipyevents
# %pip install trame
# %pip install trame-vuetify
# %pip install trame-vtk
# %pip install pyvista

import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
import torch.nn as nn
from mne import setup_source_space
from mne import make_forward_solution
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import fetch_fsaverage
from mne import compute_raw_covariance

# Creating a MNE Raw object
ch_names = ['EEG {0}'.format(i) for i in range(64)] 
ch_types = ['eeg' for _ in range(64)]
info = mne.create_info(ch_names=ch_names, sfreq=160.0, ch_types=ch_types)


raw = mne.io.RawArray(np.load("./data/training/normalized-training-open-64ch.npy")[0], info)
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
new_names = ten_twenty_montage.ch_names[:len(raw.ch_names)]
mapping = dict(zip(raw.ch_names, new_names)) # Create a dictionary mapping old names to new names then rename
raw.rename_channels(mapping)
raw.set_montage(ten_twenty_montage)
raw.set_eeg_reference('average', projection=True)


# Use pre-processing methods in MNE, for instance: filter the signal
raw.filter(l_freq=1, h_freq=None)

# Setting the subjects_dir path
subjects_dir = str(mne.datasets.sample.data_path()) + '/subjects'

# Setting up the source space
src = setup_source_space('fsaverage', spacing='oct6', add_dist=False, subjects_dir=subjects_dir)



# Making the forward model
fsaverage = mne.datasets.fetch_fsaverage(verbose=True)
bem_dir = os.path.join(fsaverage, 'bem')
bem_fname = os.path.join(bem_dir, 'fsaverage-5120-5120-5120-bem-sol.fif')
bem_sol = mne.read_bem_solution(bem_fname, verbose=True)

# Now pass the bem_sol to the make_forward_solution function
fwd = make_forward_solution(raw.info, trans=None, src=src, bem=bem_sol, meg=False, eeg=True)



# Compute the inverse solution
noise_cov = compute_raw_covariance(raw, tmin=0.0, tmax=9760 / raw.info['sfreq']) # Compute the covariance on a segment of the raw data
inv = make_inverse_operator(raw.info, fwd, noise_cov, loose=0.2, depth=0.8)
lambda2 = 1.0 / 9.0  # this is equivalent to using a signal-to-noise ratio of 3
method = 'dSPM'  # use dSPM method (could also be MNE or sLORETA)
events = mne.make_fixed_length_events(raw, duration=1.0)  # Here duration is in sec.
epochs = mne.Epochs(raw, events, tmin=0, tmax=10, baseline=None, reject=None)  # Here tmin and tmax are in sec.
evoked = epochs.average()



stc = apply_inverse(evoked, inv, lambda2, method)
stc.plot(
    subject='fsaverage',
    hemi='both',  # Specify 'both' to show both hemispheres
    surface='inflated',
    subjects_dir=subjects_dir,
    time_viewer=True
)



Creating RawArray with float64 data, n_channels=64, n_times=3152
    Range : 0 ... 3151 =      0.000 ...    19.694 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 529 samples (3.306 s)

Setting up the source space with the following parameters:

SUBJECTS_DIR = C:\Users\joshua.park\mne_data\MNE-sample-data\subjects
Subject      = fsaverage
Surface      = white
Octahedron subdivision 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s



>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading C:\Users\joshua.park\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.white...
Mapping lh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\joshua.park\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/163842 selected to source space (oct = 6)

Loading C:\Users\joshua.park\mne_data\MNE-sample-data\subjects\fsaverage\surf\rh.white...
Mapping rh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\joshua.park\mne_data\MNE-sample-data\subjects\fsaverage\surf\rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white 4098/163842 selected to source space (oct = 6)

You are now one step closer to computing the gain matrix
0 files missing from root.txt in C:\Users\joshua.park\mne_data\MNE-sam

In [23]:
# import mne
# from mne.datasets import sample
# from mne import make_forward_solution
# from mne.minimum_norm import make_inverse_operator, apply_inverse

# # Load raw data
# raw = mne.io.read_raw_edf('./data/training/EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R01.edf', preload=True)

# # Preprocessing with ICA
# raw.filter(l_freq=1, h_freq=79)  # bandpass filter
# ica = mne.preprocessing.ICA(n_components=20, random_state=123, method='fastica')
# ica.fit(raw) 
# raw = ica.apply(raw)  # Apply ICA to raw data

# # Creating epochs
# epochs = mne.make_fixed_length_epochs(raw, duration=2.0) # 2-second epochs

# # Preparing for source localization
# info = epochs.info

# # Paths for 'fsaverage' source space and BEM files
# subjects_dir = str(mne.datasets.sample.data_path()) + '/subjects'
# subject = 'fsaverage'
# trans = 'fsaverage'  # fsaverage does not require a transformation matrix
# src = subjects_dir + '/fsaverage/bem/fsaverage-ico-5-src.fif'
# bem = subjects_dir + '/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif'

# # adjusted standard channel names
# standard_ch_names = ['Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F3', 
#                      'Fz', 'F4', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4',
#                      'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 
#                      'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
#                      'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7']

# # Remove any existing montage
# raw.set_montage(None)

# # Rename the raw object's channel names to match the 10-20 system
# rename_dict = {old: new for old, new in zip(raw.ch_names, standard_ch_names)}
# raw.rename_channels(rename_dict)

# # Now you should be able to set the montage
# montage = mne.channels.make_standard_montage('standard_1020')
# raw.set_montage(montage)


# # Making the Forward solution
# fwd = make_forward_solution(info, trans, src, bem, eeg=True, mindist=5.0, n_jobs=1)
# # Computing the covariance matrix
# cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'])
# # Creating the inverse operator
# inverse_operator = make_inverse_operator(info, fwd, cov, loose=0.2, depth=0.8)

# # Defining the inverse parameters
# method = "eLORETA"
# snr = 3.
# lambda2 = 1. / snr ** 2
# # Extracting the evoked response and applying the inverse
# evoked = epochs.average()
# stc = apply_inverse(evoked, inverse_operator, lambda2, method=method, pick_ori=None)

# # Plotting the source estimates
# stc.plot(subject=subject, subjects_dir=subjects_dir, initial_time=0.1, hemi='both')


Extracting EDF parameters from c:\Users\joshua.park\Desktop\eeg-wgan\data\training\EEGData\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 79 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 79.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 79.50 Hz)
- Filter length: 529 samples (3.306 s)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 20 components


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.2s.
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


ValueError: DigMontage is only a subset of info. There are 14 channel positions not present in the DigMontage. The channels missing from the montage are:

['Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.